In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
from tensorflow.keras.models import load_model
import pickle

In [3]:
# Load the trained model 
model=load_model('first.h5')

# Load the scaler and encoder
with open("label_enc_gender.pkl",'rb') as file :
    label_enc_gen=pickle.load(file)

with open("one_hot_geo.pkl",'rb') as file :
    one_hot_geo=pickle.load(file)
    
with open("scaler.pkl",'rb') as file :
    scaler=pickle.load(file)


In [4]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [14]:
# Method 1 of application of encoder 
# Directly applying on dict

# One-hot encode 'Geography'
geo_encoded = one_hot_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=one_hot_geo.get_feature_names_out(['Geography']))
geo_encoded_df


/Users/kishan/Desktop/python/env/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [15]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [16]:
# Method 2 
## Encode categorical variables
input_df['Gender']=label_enc_gen.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [17]:
## concatination one hot encoded 
input_df=pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [18]:
## Scaling the input data
input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.52794372,  0.91634307,  0.10714725, -0.69485692, -0.26543272,
         0.79175325,  0.64109738,  0.96572973, -0.87279329,  0.98807114,
        -0.57449099, -0.57097173]])

In [19]:
## PRedict churn
prediction=model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


array([[0.04264887]], dtype=float32)

In [20]:
prediction_proba = prediction[0][0]
prediction_proba

0.042648874

In [21]:
if prediction_proba > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.
